In [1]:
from modules import abrindo_dados, adicionando_dados, caixa_de_ferramentas_vetores, pos_analise_de_dados
import os
import geopandas as gpd
import pandas as pd

/home/luisthethormes/sol_geo_env/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
#Abrindo arquivos
vault_files = "/home/luisthethormes/01_sol/geo-dev/Sol-luis.github.io/data/"

limite_produtor = abrindo_dados.process_files(vault_files, "pol_props*", "geojson", zipped = False)
limite_produtor = limite_produtor[['car','produtor', 'municipio', 'ind_status', 'des_condic', 'area_hectares', 'geometry']]
desmates = abrindo_dados.process_files(vault_files, "prodes_deforestation_*", "geojson", zipped = False)
desmates = desmates[['year', 'image_date', 'geometry']]
es_uso_solo_vetor = abrindo_dados.process_files(vault_files, "es_uso_solo_vetor_reclassificado_pt*", "geojson", zipped = False)

### períodos de análise do desmatamento
years_cols_pos_2008 = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,2017, 2018, 2019, 2020, 2021, 2022, 2023]
years_cols_pos_2021 = [2021, 2022, 2023]   

Dimensões do(s) arquivo(s) ['pol_props_ES.geojson']: (73, 15)
Dimensões do(s) arquivo(s) ['prodes_deforestation_clip.geojson']: (60, 16)
Dimensões do(s) arquivo(s) ['es_uso_solo_vetor_reclassificado_pt.geojson']: (10948, 3)


### Análise de desmates & uso do solo

In [3]:
#Unindo espacialmente
limite_produtor_left_desmates = caixa_de_ferramentas_vetores.spatial_intersection(desmates, limite_produtor)
limite_produtor_left_uso_solo = caixa_de_ferramentas_vetores.spatial_intersection(es_uso_solo_vetor, limite_produtor)

crs are the same
crs are different


In [4]:
limite_produtor_left_uso_solo = limite_produtor_left_uso_solo.rename(columns={'area_hectares': 'area_propriedade_hectares'}, inplace=False)
limite_produtor_left_desmates = limite_produtor_left_desmates.rename(columns={'area_hectares': 'area_propriedade_hectares'}, inplace=False)

In [5]:
### Calculando área em hectares ###
limite_produtor_left_desmates = adicionando_dados.add_area(limite_produtor_left_desmates)
limite_produtor_left_uso_solo = adicionando_dados.add_area(limite_produtor_left_uso_solo)

In [6]:
# limite_produtor_left_desmates[['area_hectares_right', 'year']] =  limite_produtor_left_desmates[['area_hectares_right', 'year']].fillna(0)
limite_produtor_left_desmates['year'] = limite_produtor_left_desmates['year'].astype(int)

In [7]:
#pivoting results by year and car while suming area_hectares
pivoted_deforestation = pd.pivot_table(limite_produtor_left_desmates,
                                        index=['car','produtor', 'municipio','area_propriedade_hectares'],
                                          columns='year',
                                            values='area_hectares',
                                              aggfunc='sum',
                                                dropna = True,
                                                  fill_value=0)
pivoted_deforestation_r = pivoted_deforestation.reset_index()
pivoted_deforestation_r[[2009, 2012, 2015, 2017, 2020, 2022]] = 0.0 ##adicionando as colunas em que não houve desmatamento

### Reordenando df
pivoted_deforestation_final = pivoted_deforestation_r[['car',
                                                        'produtor',
                                                          'municipio',
                                                            'area_propriedade_hectares',
                                                             2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
                                                             2017, 2018, 2019, 2020, 2021, 2022, 2023]]
### Adding
limite_produtor_merge = limite_produtor[['car', 'produtor', 'municipio', 'area_hectares']]
pivoted_deforestation_final_merge  = pivoted_deforestation_final[['car', 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
                                                             2017, 2018, 2019, 2020, 2021, 2022, 2023]]

limite_desmatamento_full = pd.merge(limite_produtor_merge, pivoted_deforestation_final_merge, how='left', on='car')
limite_desmatamento_full = limite_desmatamento_full.fillna(0)

pivoted_deforestation_final = limite_desmatamento_full.rename (columns={'area_propriedade_hectares' : 'area da propriedade rural (ha)', 
                                                                            'car': 'código CAR'                                                                           
                                                                           })
pivoted_deforestation_final.to_excel(os.path.join(vault_files, 'Análise desmatamento série (2008-2023).xlsx'), index =  False)

In [8]:
#retirando duplicatas
pivoted_deforestation_final = pivoted_deforestation_final.drop_duplicates(subset='código CAR', keep='last')
#retirando cancelados sobrepostos
list_overlap = ['ES-3203056-78C1872E88A04F8F88A49DF118230B2C', 'ES-3203056-BBC9D63224A74A8983B475773884AEBD']
pivoted_deforestation_final = pivoted_deforestation_final[~pivoted_deforestation_final['código CAR'].isin(list_overlap)]

In [9]:
pivoted_deforestation_final.groupby(by='municipio').sum().reset_index().to_excel(os.path.join(vault_files, 'Análise desmatamento nível Municipal.xlsx'), index=False)

/tmp/ipykernel_136197/3011093000.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivoted_deforestation_final.groupby(by='municipio').sum().reset_index().to_excel(os.path.join(vault_files, 'Análise desmatamento nível Municipal.xlsx'), index=False)


In [10]:
pivoted_deforestation_final

list_count_deforestation_per_year =  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
                                                             2017, 2018, 2019, 2020, 2021, 2022, 2023
                                                             ]

In [11]:
# stats = pos_analise_de_dados.calculate_stats(pivoted_deforestation_final, stats=['mean', 'median', 'std', 'max', 'sum'])
# stats = stats.rename(columns={
#             'mean': 'Média',
#             'median': 'Mediana',
#             'std': 'Desvio padrão',
#             'max': 'Máximo',
#             'sum': 'Soma',
#         })

# stats.to_excel(os.path.join(vault_files, 'Estatísticas desmatamento série (2008-2023).xlsx'), index=False)

In [12]:
limite_produtor_left_uso_solo = pd.pivot_table(limite_produtor_left_uso_solo,
                                        index=['car','produtor', 'municipio','area_propriedade_hectares'],
                                          columns='classe_uso_solo_mapbiomas',
                                            values='area_hectares',
                                              aggfunc='sum',
                                                  fill_value=0)                                                
limite_produtor_left_uso_solo_r = limite_produtor_left_uso_solo.reset_index()
limite_produtor_left_uso_solo_r.to_excel(os.path.join(vault_files, 'Análise uso do solo série(2023).xlsx'), index=False)

In [15]:
pivoted_uso_solo_final = limite_produtor_left_uso_solo_r.drop_duplicates(subset='car', keep='last')
#retirando cancelados sobrepostos
list_overlap = ['ES-3203056-78C1872E88A04F8F88A49DF118230B2C', 'ES-3203056-BBC9D63224A74A8983B475773884AEBD']
pivoted_uso_solo_final = pivoted_uso_solo_final[~pivoted_uso_solo_final.car.isin(list_overlap)]
pivoted_uso_solo_final.groupby(by='municipio').sum().reset_index().to_excel(os.path.join(vault_files, 'Uso do Solo nível municipal.xlsx'), index= False)

/tmp/ipykernel_136197/3561316865.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivoted_uso_solo_final.groupby(by='municipio').sum().reset_index().to_excel(os.path.join(vault_files, 'Uso do Solo nível municipal.xlsx'), index= False)


In [17]:
pivoted_uso_solo_final.groupby(by='municipio')['municipio'].count().to_excel(os.path.join(vault_files, 'Contagem propriedades por Municipio.xlsx'), index=False)

In [83]:
# stats_uso_solo = pos_analise_de_dados.calculate_stats(limite_produtor_left_uso_solo_r, stats=['mean', 'median', 'std', 'max', 'sum'])
# stats_uso_solo.to_excel(os.path.join(vault_files, 'Estatísticas uso do solo série (2023).xlsx'), index=False)

### Análise dos status das propriedades 

In [29]:
#Criando % de propriedades por cada estatus
limite_produtor = limite_produtor.drop_duplicates(subset = 'car', keep= 'first')
df = limite_produtor.des_condic.value_counts()/int(len(limite_produtor)) * 100
df = (pd.DataFrame(df)).reset_index()
df = df.rename({'index' : 'Status', 
                'des_condic': 'Porcentagem'
                }, axis =1)

In [30]:
limite_produtor_por_municipio = limite_produtor.groupby(['municipio', 'des_condic']).size().reset_index(name = "contagem")
### Calculando % por municipio
soma_total = limite_produtor_por_municipio.contagem.sum()

# Calcula a soma da contagem por município
soma_por_municipio = limite_produtor_por_municipio.groupby('municipio')['contagem'].transform('sum')

# Adiciona a coluna de porcentagem em relação ao município
limite_produtor_por_municipio['% do município'] = (limite_produtor_por_municipio['contagem'] / soma_por_municipio) * 100

# Adiciona a coluna de porcentagem em relação ao total da amostra
limite_produtor_por_municipio['% do total amostrado'] = (limite_produtor_por_municipio['contagem'] / soma_total) * 100

limite_produtor_por_municipio


,municipio,des_condic,contagem,% do município,% do total amostrado
0,Afonso Claudio,Aguardando analise,8,100.000000,11.111111
1,Aracruz,Aguardando analise,1,100.000000,1.388889
2,Boa Esperanca,Aguardando analise,1,100.000000,1.388889
3,Ecoporanga,Aguardando analise,1,100.000000,1.388889
4,Fundao,Aguardando analise,3,100.000000,4.166667
5,Governador Lindenberg,Aguardando analise,1,100.000000,1.388889
6,Itaguacu,Aguardando analise,7,100.000000,9.722222
7,Itarana,Aguardando analise,1,100.000000,1.388889
8,Jaguare,Aguardando analise,14,77.777778,19.444444
9,Jaguare,Cancelado por decisao administrativa,4,22.222222,5.555556


In [31]:
### Exporting in one file df and limite_produtor_por_municipio using xlsx writer

with pd.ExcelWriter(os.path.join(vault_files, 'Análises de Status das propriedades.xlsx'), engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name="Análise geral", index=False)
    limite_produtor_por_municipio.to_excel(writer, sheet_name="Análise por município", index=False)